In [20]:
import numpy as np
import math
import qinfer
from collections import deque

## Define the gate set

### Helper functions for making common gates


In [21]:
def S_gate(dim):
    if dim == 2:
        return np.matrix(np.diag([1,1j]))
    omega = np.exp(1j*2*np.pi/dim)
    S_gate = np.zeros((dim,dim),dtype=complex)
    for i in range(dim):
        if dim % 2 == 0:
            S_gate[i,i] = omega**(i*(i)/(2))
        else:
            S_gate[i,i] = omega**(i*(i+1)/(2))
    return S_gate

def H_gate(dim):
    if dim == 2:
        return 1/np.sqrt(2)*np.matrix([[1,1],[1,-1]])
    omega = np.exp(1j*2*np.pi/dim)
    H = np.zeros((dim,dim),dtype=complex)
    for i in range(dim):
        for j in range(dim):
            H[i,j] = 1/math.sqrt(dim) * omega**(j*i)
    return H

def Z_gate(dim):
    omega = np.exp(1j*2*np.pi/dim)
    return np.matrix(np.diag([omega**n for n in range(dim)]))

In [22]:
np.round(Z_gate(4),4)

matrix([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+1.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, -0.-1.j]])

In [27]:
S = [S_gate(6),H_gate(6)]

## Finding Adjoint representation in SO(d^2-1)

Generators of $\mathfrak{s u}(d)$ are found in mathematica via the approach found in https://mathematica.stackexchange.com/questions/159014/calculate-representations-of-sun-generators



In [29]:
dim = 6
B = qinfer.tomography.gell_mann_basis(dim)
G = []
for i in range(1,len(B)):
    G.append(np.matrix(B[i].full()))


## Determine if the center of the subgroup is the trivial center

In [30]:
# generate the adjoint of an operator in SU(d) in SO(d^2-1)
def Ad(U,G):
    d = len(G)
    Ad_U = np.matrix(np.zeros((d,d),dtype=complex))
    for i in range(d):
        for j  in range(d):
            Ad_U[i,j] = -1/2 * np.trace(G[i]*U*G[j]*np.linalg.inv(U))
    return Ad_U

def check_Center(S,G):
    dim = len(G)
    I = np.matrix(np.eye(dim))
    Ms = np.matrix(np.zeros((dim**2*len(S),dim**2),dtype=complex))
    for i,gate in enumerate(S):
        Ms[i*dim**2:(i+1)*dim**2,:] = np.kron(I,Ad(gate,G)) - np.kron(Ad(np.conj(gate.T),G),I)
        
    dim_ker = len(G)**2 - np.linalg.matrix_rank(Ms)
    return dim_ker == 1

In [32]:
check_Center(S,G)
Ad(np.eye(6),G)

matrix([[-0.5+0.j, -0. +0.j, -0. +0.j, ..., -0. +0.j, -0. +0.j, -0. +0.j],
        [-0. +0.j, -0.5+0.j, -0. +0.j, ..., -0. +0.j, -0. +0.j, -0. +0.j],
        [-0. +0.j, -0. +0.j, -0.5+0.j, ..., -0. +0.j, -0. +0.j, -0. +0.j],
        ...,
        [-0. +0.j, -0. +0.j, -0. +0.j, ..., -0.5+0.j, -0. +0.j, -0. +0.j],
        [-0. +0.j, -0. +0.j, -0. +0.j, ..., -0. +0.j, -0.5+0.j, -0. +0.j],
        [-0. +0.j, -0. +0.j, -0. +0.j, ..., -0. +0.j, -0. +0.j, -0.5+0.j]])

## Determine if the Subgroup is infinite or finite

In [14]:
def ball_check(gate,N):
    dim = gate.shape[0]
    alphas = [np.exp(1j*2*np.pi/dim*m) for m in range(6)]
    nth_gate = gate
    for n in range(N):
        trace = np.trace(nth_gate)
        for alpha in alphas:
            if (2*(dim) - alpha*np.conj(trace) - np.conj(alpha)*trace <= 1/2): # check if it is in B
                if not np.allclose(nth_gate/nth_gate[0,0],np.eye(dim)): #check if it is in the center
                    print(nth_gate)
                    return True
        nth_gate = nth_gate @ gate # increase the power by one
    return False


def ball_check_eigs(gate,N):
    dim = gate.shape[0]
    eig,_ = np.linalg.eig(gate) 
    for n in range(N):
        for m in range(dim):
            theta = 2*np.pi*m/dim
            eig_sum = 0
            for i in range(dim):
                phi = n*np.angle(eig[i])
                eig_sum += np.sin((phi-theta)/2)**2
            if eig_sum < 1/8:
                if not np.allclose(eig**n,eig[0]**n*np.ones(dim)): # make sure all the eigenvalues arent the same
                    return True
    return False


In [15]:
def test_close(A,B):
    dim = B.shape[0]
    return math.isclose(np.abs(np.trace(np.conj(A.T)@B)),dim,rel_tol=1e-2)


def add_unique_bool(new_elems, group_elems):
    added = False
    for new_elem in new_elems:
        flag = False
        for group_elem in group_elems:
            if test_close(new_elem,group_elem):
                flag = True
                break
        if not(flag): 
            group_elems.append(new_elem)
            added = True
    return added

def add_unique_num(new_elems, group_elems):
    added = 0
    for new_elem in new_elems:
        flag = False
        for group_elem in group_elems:
            if test_close(new_elem,group_elem):
                flag = True
                break
        if not(flag): 
            group_elems.append(new_elem)
            added += 1
    return added

In [16]:
def check_Finite(S,N,lmax):
    dim = S[0].shape[0]
    G_s = [np.matrix(np.eye(dim))]
    for l in range(lmax):# check words up to length 10 starting at l = 0
        new_gates = []
        for gate in G_s: 
            if ball_check(gate,N):
                print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
                return [False,-1]
            for U in S:
                new_gates.append(gate@U) # might be adding duplicate elements 
        print('l = ',l)
        if add_unique_bool(new_gates,G_s) == 0:
            return [True,len(G_s)]
    return [True,len(G_s)]


def check_Finite_eigs(S,N,lmax):
    dim = S[0].shape[0]
    G_s = [np.matrix(np.eye(dim))]
    new_index = 0
    next_index = 1
    for l in range(lmax):# check words up to length lmax starting at l = 0
        new_gates = deque()
        for gate in G_s[new_index:]: 
            if ball_check_eigs(gate,N):
                print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
                return [False,-1]
            for U in S:
                new_gates.append(gate@U) 
      
        num_added = add_unique_num(np.array(new_gates),G_s)
        print('l = ',l, 'current size = ', len(G_s))
        new_index = next_index
        next_index += num_added 
        if num_added == 0:
            return [True,len(G_s)]
    return [True,len(G_s)]

In [204]:
check_Finite_eigs(S,10,13)

l =  0 current size =  3
l =  1 current size =  7
l =  2 current size =  14
l =  3 current size =  26
l =  4 current size =  46
l =  5 current size =  75
l =  6 current size =  114
l =  7 current size =  160
l =  8 current size =  197
l =  9 current size =  212
l =  10 current size =  216
l =  11 current size =  216


[True, 216]

In [151]:
a = [1,2,3]
a.append(4,5,6)
a

TypeError: list.append() takes exactly one argument (3 given)

## Example: Finding the dimension of the Clifford group for qudits
We expect that the dimension should be $d^3(d^2-1)$

In [233]:
dim = 6
S = [S_gate(dim),H_gate(dim),Z_gate(dim)]
a,number = check_Finite_eigs(S,10,20)
print('calculated = ', number )

l =  0 current size =  4
l =  1 current size =  12
l =  2 current size =  32
l =  3 current size =  79
l =  4 current size =  183
l =  5 current size =  396
l =  6 current size =  792
l =  7 current size =  1452
l =  8 current size =  2392
l =  9 current size =  3519
l =  10 current size =  4551
l =  11 current size =  5092
l =  12 current size =  5183
l =  13 current size =  5184
l =  14 current size =  5184
calculated =  5184


In [164]:
def check_universal(S,N=10,lmax=100):
    dim = S[0].shape[0]
    B = qinfer.tomography.gell_mann_basis(dim)
    G = []
    for i in range(1,len(B)):
        G.append(np.matrix(B[i].full()))
    center = check_Center(S,G)
    if not center:
        return False
    ball,number = check_Finite_eigs(S,N,lmax)
    return ball

l =  0 current size =  4
l =  1 current size =  10
Infinite [[ 0.70710678+0.j   0.5       +0.5j]
 [ 0.70710678+0.j  -0.5       -0.5j]]
calculated =  -1


In [189]:
for i,gate in enumerate(number):
    for j,gate_2 in enumerate(number):
        if i !=j:
            if np.abs(np.trace(np.conj(gate.T)@gate_2))> 3.2:
                print(np.abs(np.trace(np.conj(gate.T)@gate_2)))

